# RILA EDA: Feature Selection with AIC - REFACTORED

**Refactored:** 2026-01-28  
**Original:** notebooks/rila/04_RILA_feature_selection.ipynb  

**Changes:**
- Migrated data loading from helpers.* to src.* imports
- Added canonical sys.path auto-detection
- Preserved AIC selection logic inline (no direct src equivalent)
- Kept position lag features inline (experiment-specific)
- Kept BaggingRegressor/Ridge ensemble inline (EDA-appropriate)
- Improved cell structure and documentation

**Purpose:** Feature selection using AIC-based exhaustive search and model comparison with bagging ensemble validation.

**Dependencies:** 03_EDA_RILA_feature_engineering.ipynb (requires engineered features)

**Note:** AIC selection logic kept inline per EDA principles - no clear src equivalent exists. This allows for exploratory flexibility in feature combination search.

## Notebook Outline
1. Setup and data loading
2. Position lag feature engineering
3. AIC-based feature selection
4. Model validation with bagging ensemble
5. Results visualization

In [ ]:
%%capture
# =============================================================================
# STANDARD SETUP CELL - Clean Dependency Pattern
# =============================================================================

# Standard library imports
import sys
import os
import re
import warnings
from datetime import datetime, timedelta
from itertools import product
from typing import Union

# Third-party imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from tqdm import tqdm

# Scikit-learn imports
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, RepeatedKFold, train_test_split
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error,
)
from sklearn.utils import resample

# Suppress warnings for clean output
warnings.filterwarnings("ignore")

# Canonical sys.path setup (auto-detect project root)
if os.path.basename(os.getcwd()) == "rila":
    # Running from notebooks/rila directory
    project_root = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
elif os.path.basename(os.getcwd()) == "notebooks":
    # Running from notebooks directory
    project_root = os.path.dirname(os.getcwd())
else:
    # Running from project root
    project_root = os.getcwd()

sys.path.insert(0, project_root)

# Refactored imports (src.* pattern)
from src.data import extraction as ext
from src.data.dvc_manager import load_dataset

# Matplotlib inline mode
%matplotlib inline

# Visualization theme
sns.set_theme(style="whitegrid", palette="deep")

print("✓ Dependencies loaded successfully")

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# AWS configuration
aws_config = {
    "xid": "x259830",
    "role_arn": "arn:aws:iam::159058241883:role/isg-usbie-annuity-CA-s3-sharing",
    "sts_endpoint_url": "https://sts.us-east-1.amazonaws.com",
    "source_bucket_name": "pruvpcaws031-east-isg-ie-lake",
    "output_bucket_name": "cdo-annuity-364524684987-bucket",
    "output_base_path": "ANN_Price_Elasticity_Data_Science",
}

# Date parameters
current_time = datetime.now()
current_date = current_time.strftime("%Y-%m-%d")
date_path = f"year={current_time.year}/month={current_time.month:02}/day={current_time.day:02}"
current_date_of_mature_data = (current_time - timedelta(days=0)).strftime("%Y-%m-%d")

# Version
version = "v2_0"

print(f"✓ Configuration loaded")
print(f"  Version: {version}")
print(f"  Current date: {current_date}")

## 1. Load Engineered Features

Load the final dataset with all engineered features from notebook 03.

In [ ]:
# =============================================================================
# LOAD DATA - Using DVC or S3 direct
# =============================================================================

# Try DVC first, fallback to S3
try:
    df_RILA = load_dataset("RILA_final_data_set", version=version)
    print("✓ Data loaded from DVC")
except Exception as e:
    print(f"DVC load failed: {e}")
    print("Attempting S3 direct load...")
    
    # AWS Connection setup
    sts_client = ext.setup_aws_sts_client_with_validation(aws_config)
    assumed_role = ext.assume_iam_role_with_validation(sts_client, aws_config)
    s3_resource, bucket = ext.setup_s3_resource_with_validation(
        assumed_role["Credentials"], aws_config["output_bucket_name"]
    )
    
    # S3 path
    file_path = f"RILA_{version}/features/{date_path}/"
    file_name = "RILA_final_data_set"
    
    # Load from S3
    df_RILA = ext.download_s3_parquet_with_optional_date_suffix(
        s3_resource, bucket, f"{file_path}{file_name}"
    )
    print("✓ Data loaded from S3")

print(f"  Shape: {df_RILA.shape}")
print(f"  Date range: {df_RILA['date'].min()} to {df_RILA['date'].max()}")

## 2. Position Lag Features

Create position lag features with exponential weighting. These capture the relative position of Prudential rates vs competitor rates over time.

**Note:** Kept inline per EDA principles - experiment-specific transformation.

In [ ]:
# =============================================================================
# DATA FILTERING AND POSITION LAG FEATURES
# =============================================================================

# Filter to non-zero sales
df_RILA = df_RILA[df_RILA["sales"] != 0]

# Exponential weighting (more weight to recent observations)
df_RILA["weight"] = [0.98 ** (len(df_RILA) - k) for k in range(len(df_RILA))]

# Position lag features: capture when Prudential rate is below competitor
for k in range(15):
    # pos_lag_k = max(0, C_lag - P_lag) captures when competitor is higher
    df_RILA[f"pos_lag_{k}"] = np.abs(
        df_RILA[f"C_lag_{k+2}"] - df_RILA[f"P_lag_{k}"]
    ) - (df_RILA[f"C_lag_{k+2}"] - df_RILA[f"P_lag_{k}"])
    
    # Scaled version: interaction with Prudential rate level
    df_RILA[f"pos_lag_scale_{k}"] = (
        df_RILA[f"pos_lag_{k}"] * df_RILA[f"P_lag_{k}"]
    )
    
    # Squared version: non-linear effect
    df_RILA[f"pos_lag_sq_{k}"] = df_RILA[f"pos_lag_{k}"] ** 2

# Time filtering
mask_time = df_RILA["date"] > pd.to_datetime("2022-05-01")
df_RILA = df_RILA[mask_time]
df_RILA = df_RILA[df_RILA["date"] < current_date_of_mature_data].dropna(
    subset="C_lag_13"
)

# Remove holidays
df_RILA = df_RILA[df_RILA["holiday"] == 0]

print(f"✓ Position lag features created")
print(f"  Filtered shape: {df_RILA.shape}")
print(f"  Observations: {len(df_RILA)}")
df_RILA[["date", "sales"]].head(10)

## 3. AIC-Based Feature Selection

**Note:** AIC selection logic kept inline - no direct equivalent in src.features.selection for this exhaustive combinatorial search approach.

This cell performs exhaustive feature search using AIC (Akaike Information Criterion) to balance model fit and complexity.

In [ ]:
# =============================================================================
# AIC SELECTION FUNCTIONS - Inline (no src equivalent)
# =============================================================================

def best_subset_ols_strict(
    df,
    new_features,
    old_features,
    target_variable="sales_forward_0",
    target_weight=None,
    lim_num=1,
):
    """
    AIC-based exhaustive feature selection.
    
    Searches through all combinations of new_features to add to old_features,
    selecting the combination with the lowest AIC.
    
    Args:
        df: DataFrame with features and target
        new_features: List of candidate feature combinations (strings like "P_lag_0+C_lag_2")
        old_features: List of features to always include
        target_variable: Target column name
        target_weight: Optional weight column
        lim_num: Number of features to add from new_features
    
    Returns:
        AIC_min_list: List of minimum AIC values
        N_obs_validation_list: List of observation counts
        old_features_iterated: Updated feature list
        df_results_all: DataFrame with all tried combinations and AICs
    """
    AIC_min_list = []
    N_obs_validation_list = []
    old_features_iterated = old_features.copy()
    df_results_all = pd.DataFrame(columns=["feature_name", "AIC", "N_obs"])
    
    for iteration in range(lim_num):
        AIC_min = np.inf
        best_feature = None
        
        # Try each new feature
        for idx, feature_combo in enumerate(new_features):
            if idx % 100 == 0:
                print(f"{idx}/{len(new_features)}")
            
            # Parse feature combination (e.g., "P_lag_0+C_lag_2")
            features_to_test = old_features_iterated + feature_combo.split("+")
            
            # Check all features exist
            if not all(f in df.columns for f in features_to_test):
                continue
            
            # Fit OLS model
            try:
                X = df[features_to_test].copy()
                X = sm.add_constant(X)
                y = df[target_variable]
                
                if target_weight:
                    weights = df[target_weight]
                    model = sm.OLS(y, X, weights=weights).fit()
                else:
                    model = sm.OLS(y, X).fit()
                
                aic = model.aic
                n_obs = model.nobs
                
                # Store result
                df_results_all = pd.concat([
                    df_results_all,
                    pd.DataFrame({"feature_name": [feature_combo], "AIC": [aic], "N_obs": [n_obs]})
                ], ignore_index=True)
                
                # Track best
                if aic < AIC_min:
                    AIC_min = aic
                    best_feature = feature_combo
                    N_obs_validation = n_obs
            
            except Exception as e:
                continue
        
        # Add best feature to old_features
        if best_feature:
            AIC_min_list.append(AIC_min)
            N_obs_validation_list.append(N_obs_validation)
            old_features_iterated.extend(best_feature.split("+"))
            
            # Refit with best features and display
            X = df[old_features_iterated].copy()
            X = sm.add_constant(X)
            y = df[target_variable]
            
            if target_weight:
                weights = df[target_weight]
                model = sm.OLS(y, X, weights=weights).fit()
            else:
                model = sm.OLS(y, X).fit()
            
            print(model.summary())
    
    return AIC_min_list, N_obs_validation_list, old_features_iterated, df_results_all

print("✓ AIC selection functions defined")

In [ ]:
# =============================================================================
# AIC FEATURE SELECTION - Exhaustive Search
# =============================================================================

# Starting features (empty for fresh search)
old_features = []

# Generate candidate feature combinations
L = 2  # Lag offset for competitor rates
new_features = []

# Combinations with C_diff (rate changes)
for j in range(4):
    new_features += [
        f"P_lag_{k}+C_lag_{k+L}+C_diff_lag_{k+L+j+1}+sales_by_contract_date_lag_{k+L+1}"
        for k in range(0, 6, 1)
    ]

# Combinations with sales lags
for j in range(4):
    new_features += [
        f"P_lag_{k}+C_lag_{k+L}+sales_by_contract_date_lag_{k+L+j+1}"
        for k in range(0, 6, 1)
    ]

# Combinations with rate changes only
for j in range(4):
    new_features += [
        f"P_lag_{k}+C_lag_{k+L}+C_diff_lag_{k+L+j+1}" for k in range(0, 6, 1)
    ]

# Base combinations (just rates)
for j in range(4):
    new_features += [f"P_lag_{k}+C_lag_{k+L}" for k in range(0, 6, 1)]

print(f"Testing {len(new_features)} feature combinations")

# Run AIC selection
AIC_min_list, N_obs_validation_list, old_features_iterated, df_results_all = (
    best_subset_ols_strict(
        df_RILA,
        new_features,
        old_features,
        target_variable="sales_forward_0",
        lim_num=1,
    )
)

print(f"\n✓ AIC selection complete")
print(f"  Minimum AIC: {AIC_min_list}")
print(f"  Selected features: {old_features_iterated}")

# Display top AIC results
df_AIC_list = (
    df_results_all[["feature_name", "AIC", "N_obs"]]
    .sort_values(by=["AIC"], ascending=True)
    .reset_index(drop=True)
)
print("\nTop 10 feature combinations by AIC:")
print(df_AIC_list.head(10))

## 4. Model Validation with Bagging Ensemble

Validate selected features using BaggingRegressor with Ridge estimators.

**Note:** Kept inline per EDA principles - experiment-specific ensemble validation approach.

In [ ]:
# =============================================================================
# BAGGING ENSEMBLE VALIDATION
# =============================================================================

df = df_RILA.copy()
df["weight"] = [0.99 ** (len(df) - k) for k in range(len(df))]
df = df.reset_index(drop=True)

# Target variables
target_variable = "sales_forward_0"
target_variable_alt = "sales_by_contract_date_lag_0"

# Selected features from AIC
features = old_features_iterated

# Spread calculation (for visualization)
df["Spread"] = df["P_lag_4"] - df["C_lag_6"]

# Prepare data
X = df[features]
y = df[target_variable_alt]
y_test_orig = df[target_variable_alt]

# Bagging setup
N_iter = 100
scores_MAPE = np.zeros(N_iter)
scores_MAPE_orig = np.zeros(N_iter)
scores_AE = np.zeros(N_iter)
scores_r2 = np.zeros(N_iter)
scores_r2_orig = np.zeros(N_iter)

# Results storage
df_results = pd.DataFrame(
    columns=[
        "date",
        "spread",
        "y_predict",
        "y_predict_orig",
        "y",
        "y_orig",
        "run",
        "quarter",
        "residual",
    ]
)

# Fit bagging model
print("Fitting bagging ensemble...")
model = BaggingRegressor(estimator=Ridge(), n_estimators=N_iter, random_state=42)
bagged_model = model.fit(df[features], df[target_variable])

# Evaluate each estimator
for k, lr_model in enumerate(bagged_model.estimators_):
    y_predict = lr_model.predict(X)
    y_predict_orig = y_predict
    
    # Store results
    df_results_temp = pd.DataFrame({
        "spread": df["Spread"],
        "y_predict_orig": y_predict_orig,
        "y_predict": y_predict,
        "residual": y - y_predict,
        "y": y,
        "y_orig": y_test_orig,
        "run": k,
        "quarter": df["quarter"],
        "date": df["date"],
    })
    df_results = pd.concat([df_results, df_results_temp], ignore_index=True)
    
    # Calculate metrics
    sample_weights = y_test_orig / y_test_orig.sum()
    scores_AE[k] = 100 * y_predict_orig.sum() / y_test_orig.sum()
    scores_MAPE_orig[k] = 100 * mean_absolute_percentage_error(
        y_test_orig, y_predict_orig, sample_weight=sample_weights
    )
    scores_MAPE[k] = 100 * mean_absolute_percentage_error(
        y, y_predict, sample_weight=sample_weights
    )
    scores_r2[k] = r2_score(y, y_predict)
    scores_r2_orig[k] = r2_score(y_test_orig, y_predict_orig)

# Display results
print(f"\n✓ Bagging validation complete")
print(f"\nR²:       {scores_r2_orig.mean():.3f}")
print(f"std:      {scores_r2_orig.std():.3f}\n")
print(f"A/E:      {scores_AE.mean():.2f}%")
print(f"std:      {scores_AE.std():.3f}\n")
print(f"MAPE:     {scores_MAPE_orig.mean():.2f}%")
print(f"std:      {scores_MAPE_orig.std():.3f}\n")

## 5. Results Visualization

Visualize model predictions vs actuals over time and vs cap rate spread.

In [ ]:
# =============================================================================
# RESULTS VISUALIZATION
# =============================================================================

figure, axes = plt.subplots(2, 1, sharex=False, sharey=False, figsize=(16, 10))
sns.set_theme(style="whitegrid", palette="deep")
figure.suptitle("FlexGuard + PruAdvisors Feature Selection Results", fontsize=16)

# Time series plot
axes[0].set_title("Predicted vs Actual Sales Over Time")
axes[0].set_xlabel("Date")
axes[0].set_ylabel("FlexGuard Sales ($)")

sns.lineplot(
    df_results,
    x="date",
    y="y_predict",
    ax=axes[0],
    color="tab:blue",
    linewidth=5,
    errorbar=("pi", 95),
    label="Predicted (95% PI)",
)
sns.scatterplot(
    data=df_results, x="date", y="y", ax=axes[0], color="k", label="Actual", alpha=0.6
)
axes[0].legend()

# Spread plot
axes[1].set_title("Sales vs Cap Rate Spread")
axes[1].set_xlabel("Cap Rate Distance to Mean Rate (bps)")
axes[1].set_ylabel("FlexGuard Sales ($)")

sns.lineplot(
    df_results,
    x="spread",
    y="y_predict",
    ax=axes[1],
    color="tab:blue",
    linewidth=5,
    errorbar=("pi", 95),
    label="Predicted (95% PI)",
)
sns.scatterplot(
    data=df_results, x="spread", y="y", ax=axes[1], color="k", label="Actual", alpha=0.6
)
axes[1].legend()

plt.tight_layout()
plt.show()

print("✓ Visualization complete")

---

## EDA Complete

**Selected Features:** See `old_features_iterated` above

**Model Performance:**
- R²: See output above
- MAPE: See output above
- A/E: See output above

**Next Steps:** 
- Proceed to 05_RILA_Time_Forward_Cross_Validation.ipynb for walk-forward validation
- Use selected features in time series forecasting